In [2]:
import modulos as md
import sys
import os
import constantes as cons
import numpy as np 
import pandas as pd
symbol='COMBOUSDT'

lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.obtiene_historial(symbol,'30m')
        data = md.estrategia_bb(data)
        resultado = md.backtesting(data)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]}")
    except Exception as falla:
        pass  



Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: DEFIUSDT



c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:304: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


RUNEUSDT - retorno: 1.64% - trades: 11
SUSHIUSDT - retorno: 3.34% - trades: 9
